In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)

from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional, Sequence

## Setting the Functions LLM

In [ ]:
functions_llm = ChatOpenAI(
    model = "gpt-3.5-turbo-0613",
    temperature = 0,
    openai_api_key = open("openai_api.txt").read()
)

## Using OpenAI functions

This walkthrough demonstrates how to incorporate OpenAI `function-calling` API's in a chain.

We'll go over:
1. How to use functions to get structured outputs from ChatOpenAI
2. How to create a generic chain that uses (multiple) functions
3. How to create a chain that actually executes the chosen function

## Getting structured outputs

We can take advantage of OpenAI functions to try and force the model to return a particular kind of structured output. We'll use create_structured_output_chain to create our chain, which takes the desired structured output either as a `Pydantic` class or as `JsonSchema`.

When passing in Pydantic classes to structure our text, we need to make sure to have a docstring description for the class. It also helps to have descriptions for each of the classes attributes.

### Using Pydantic Classes

In [ ]:
class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for extracting information in structured formats."),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

chain = create_structured_output_chain(
    output_schema = Person,
    llm = functions_llm,
    prompt = prompt,
    verbose = True
)

In [ ]:
chain.run("Sally is 13")

To extract arbitrarily many structured outputs of a given format, we can just create a wrapper Pydantic class that takes a sequence of the original class.

In [ ]:
class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")


chain = create_structured_output_chain(
    output_schema = People,
    llm = functions_llm,
    prompt = prompt,
    verbose = True
)

In [ ]:
chain.run("Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.")

### Using JSONSchema

We can also pass in JsonSchema instead of Pydantic classes to specify the desired structure. When we do this, our chain will output JSON corresponding to the properties described in the JsonSchema, instead of a Pydantic class.

In [ ]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}

In [ ]:
chain = create_structured_output_chain(
    output_schema = json_schema,
    llm = functions_llm,
    prompt = prompt,
    verbose = True
)

In [ ]:
chain.run("Sally is 13")

## Generic Functions Chain

To create a generic OpenAI functions chain, we can use the create_openai_fn_chain method. This is the same as create_structured_output_chain except that instead of taking a single output schema, it takes a sequence of function definitions.

Functions can be passed in as:
* `dicts` conforming to OpenAI functions spec,
* `Pydantic` classes, in which case they should have docstring descriptions of the function they represent and descriptions for each of the parameters,
* `Python` functions, in which case they should have docstring descriptions of the function and args, along with type hints.

### Using Pydantic classes

In [ ]:
class RecordPerson(BaseModel):
    """Record some identifying information about a pe."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")


class RecordDog(BaseModel):
    """Record some identifying information about a dog."""

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for recording entities."),
        ("human", "Make calls to the relevant function to record the entities in the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

chain = create_openai_fn_chain(
    functions = [RecordPerson, RecordDog],
    llm = functions_llm,
    prompt = prompt,
    verbose = True
)

In [ ]:
chain.run("Harry was a chubby brown beagle who loved chicken")

### Using Python functions

In [ ]:
class OptionalFavFood(BaseModel):
    """Either a food or null."""

    food: Optional[str] = Field(
        None,
        description="Either the name of a food or null. Should be null if the food isn't known.",
    )


def record_person(name: str, age: int, fav_food: OptionalFavFood) -> str:
    """Record some basic identifying information about a person.

    Args:
        name: The person's name.
        age: The person's age in years.
        fav_food: An OptionalFavFood object that either contains the person's favorite food or a null value. Food should be null if it's not known.
    """
    return f"Recording person {name} of age {age} with favorite food {fav_food.food}!"


In [ ]:
chain = create_openai_fn_chain(
    functions = [record_person],
    llm = functions_llm,
    prompt = prompt,
    verbose = True
)

In [ ]:
chain.run(
    "The most important thing to remember about Tommy, my 12 year old, is that he'll do anything for apple pie."
)

Accordingly you can also perform `Question Answering With Structure`, `Tagging`, and `Text Extractions` using OpenAI's Functions.